<a href="https://colab.research.google.com/github/Jijnash22/multimodal-parkinsons-ai/blob/main/notebooks/04_multimodal_fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
from tensorflow.keras.models import load_model

face_model = load_model(BASE + "/models/face_pd_classifier.keras")
PD_face = face_model.predict(face_emb).flatten()

print("Face samples:", len(PD_face))


ValueError: File not found: filepath=/content/drive/MyDrive/IIIT_Nagpur_PD_Project/models/face_pd_classifier.keras. Please ensure the file is an accessible `.keras` zip file.

In [4]:
!ls "/content/drive/MyDrive/IIIT_Nagpur_PD_Project/models"


face_embedding_model.keras  voice_model_32D.keras	voice_model.keras
face_model.keras	    voice_model.h5
face_pd_classifier.keras    voice_model_improved.keras


In [5]:
from tensorflow.keras.models import load_model

face_model = load_model(BASE + "/models/face_pd_classifier.keras")
PD_face = face_model.predict(face_emb).flatten()

print("Face samples:", len(PD_face))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Face samples: 40


In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

PDv = scaler.fit_transform(PD_voice.reshape(-1,1)).flatten()
PDf = scaler.fit_transform(PD_face.reshape(-1,1)).flatten()


In [7]:
y_voice = np.array([0]* (len(PDv)//2) + [1]* (len(PDv)//2))


In [8]:
y_face = np.array([0]*20 + [1]*20)


In [9]:
y_hand = np.array([0,0,0,0,1,0,0,1])  # P1,P2,P3,P4,P5,P6,P8,P9


In [11]:
PD_hand = pd.read_csv(BASE + "/features/hand_pd_scores.csv")["PD_score"].values
PD_hand


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/IIIT_Nagpur_PD_Project/features/hand_pd_scores.csv'

In [12]:
!find "/content/drive/MyDrive" -type f | grep hand_pd


/content/drive/MyDrive/IIIT_Nagpur_PD_Project/features/hand_pd_scores.csv


In [14]:
PD_hand = pd.read_csv("/content/drive/MyDrive/IIIT_Nagpur_PD_Project/features/hand_pd_scores.csv")["PD_score"].values
PD_hand


array([0.        , 0.52072478, 0.35071088, 0.3687345 , 1.        ,
       0.6197075 , 0.298603  , 0.9534077 ])

In [16]:
import glob, os

hc_files = glob.glob(BASE + "/data/voice_dataset/HC/HC_AH/*.wav")
pd_files = glob.glob(BASE + "/data/voice_dataset/PD/PD_AH/*.wav")

y_voice = np.array([0]*len(hc_files) + [1]*len(pd_files))
len(hc_files), len(pd_files), len(y_voice), len(PDv)


(41, 40, 81, 81)

In [17]:
from sklearn.metrics import roc_auc_score

results = {}

results["Voice only"] = roc_auc_score(y_voice, PDv)
results["Face only"]  = roc_auc_score(y_face, PDf)
results["Hand only"]  = roc_auc_score(y_hand, PD_hand)

results["Voice + Face"] = roc_auc_score(y_face, (PDv[:40] + PDf)/2)
results["Voice + Hand"] = roc_auc_score(y_hand, (PDv[:8] + PD_hand)/2)
results["Face + Hand"]  = roc_auc_score(y_hand, (PDf[:8] + PD_hand)/2)
results["All 3"] = roc_auc_score(y_hand, (PDv[:8] + PDf[:8] + PD_hand)/3)

results


{'Voice only': np.float64(0.901829268292683),
 'Face only': np.float64(0.5),
 'Hand only': np.float64(1.0),
 'Voice + Face': np.float64(0.6950000000000001),
 'Voice + Hand': np.float64(1.0),
 'Face + Hand': np.float64(1.0),
 'All 3': np.float64(1.0)}